# Bayesian Network

This workbook explores the concept of **bayesian networks** and **probabilistic reasoning** in Python. 

Throughout this notebook, you will use pgmpy, one of the most popular python library to work with [probabilistic graphical models](https://en.wikipedia.org/wiki/Graphical_model).You will find some guided examples to aid your understanding, and some exercises for you to implement on your own.

#### Content:
* [pgmpy](#pgmpy)
    * [Getting started](#pgmpy-start)
    * [Exercise 1 - easy](#pgmpy-ex1)
    * [Exercise 2 - moderate](#pgmpy-ex2)

## pgmpy<a class="anchor" id="pgmpy"></a>

**[pgmpy](https://pgmpy.org/)** is a python library for creating, visualise and query probabilistic graphical models (PGM), offering tools for Bayesian networks and Markov networks modeling.

In what follows, we will use the library specifically to work with Bayesian networks, which are part of this module learning. However, the library provides a whole range of functionalities for implementing probabilistic tasks on PGMs.

Let's start!

### Getting started <a class="anchor" id="pgmpy-start"></a>

In [1]:
# uncomment the cells below to install pgmpy
# ! pip install --upgrade pip
# ! pip install pgmpy

In [ ]:
# We'll access these functions throughout the notebook, to better undertstand what they refer to and when to use them
# In an actual project, you might want to import them into python at the beginning

# from pgmpy.models import BayesianNetwork
# from pgmpy.factors.discrete import TabularCPD
# from pgmpy.inference import BeliefPropagation

import networkx as nx
import matplotlib.pyplot as plt 

During Week 5 lecture we have introduced the concept of Bayesian Network using the **burglar alarm case study**:



>_You have a new burglar alarm installed at home. It is fairly reliable at detecting a burglary, but is occasionally
set off by minor earthquakes. You also have two neighbors, John and Mary, who have promised to call
you at work when they hear the alarm. John nearly always calls when he hears the alarm, but
sometimes confuses the telephone ringing with the alarm and calls then, too. Mary, on the
other hand, likes rather loud music and often misses the alarm altogether._

(Case study synopsis from Chapter 13 of _Russell, Norvig – Aritificial Intelligence, a Modern approach (2021)_)


The specific components of the network can be visualised as below:

<figure>
<img src="alarm_network.png" alt="BN network" style="width: 500px;"/>
<figcaption style="text-align:center;font-style:italic">(from Russell, Norvig – Aritificial Intelligence, a Modern approach (2021)) </figcaption>    
</figure>
 
 
Let's implement the network using pgmpy.

We first start with creating the network structure.

In [ ]:
from pgmpy.models import BayesianNetwork

In [ ]:
# Create network structure, add nodes and edges
bn_structure = [
    ("Burglary", "Alarm"),
    ("Earthquake", "Alarm"),
    ("Alarm", "JohnCalls"),
    ("Alarm", "MaryCalls")
]

alarm_model = BayesianNetwork(bn_structure)

In [ ]:
print('Model:', alarm_model)
print('Nodes:', alarm_model.nodes())
print('Edges:', alarm_model.edges())

We can use NetworkX library to plot the network

In [ ]:
# Plot the Bayesian network

nx.draw(alarm_model, 
        with_labels=True, 
        node_size=2000, node_color = 'lightblue',
        font_size=10, 
        arrows = True, arrowstyle='->', arrowsize=15,
        pos=nx.circular_layout(alarm_model))
plt.show()

Let's now add the Conditional Probability Distributions (CPD), in other words the probability tables associated to each node. In pgmpy this is done using `TabularCPD`. The parameters needed are:

- `variable`: the node name to which the table is associated.
- `variable_card`: number of states of the variable. In our case is 2 (true/false).
- `values`: conditional probabilities associated to the specific variable/node.
- `evidence`: list of variables w.r.t. the conditional probabilities are defined.
- `evidence_card`:  number of states of the evidence nodes.

The node 'Burglary' and 'Earthquake' have _unconditional_ probabilities, in other words their true/false state does not depend from any other event/evidence.

In [ ]:
from pgmpy.factors.discrete import TabularCPD

# burglary probability
# False=0.999, True=0.001
cpd_burglary = TabularCPD(
    variable="Burglary", 
    variable_card=2, 
    values=[[0.999], [0.001]]
)

# earthquake probability
# False=0.998, True=0.002
cpd_earthquake = TabularCPD(
    variable="Earthquake", 
    variable_card=2, 
    values=[[0.998], [0.002]]
)

The node 'Alarm' is conditional on 'Burglary' and 'Earthquake', hence the True/False probability state depends on the True/False state of 'Burglary' and 'Earthquake'. Looking at the above picture, we can deduce the following table of possibilities (and associated probabilities):

||||||
|---------------|-------------------|------------------|-------------------|------------------|
|               | **Burglary(False)**  | **Burglary(False)**  | **Burglary(True)**    | **Burglary(True)**  |
|               | **Earthquake(False)** | **Earthquake(True)** | **Earthquake(False)** | **Earthquake(True)** |
| **Alarm (False)** | 0.999             | 0.71             | 0.06              | 0.05             |
| **Alarm(True)**   | 0.001             | 0.29             | 0.94              | 0.95             |

Note that all columns sum up to 1. Infact, we have used the _main axiom of probability_, which states that all probabilities for an event sum up to 1, hence `P(Alarm=True)=1-P(Alarm=False)`.

Let's create such table in pmgpy.

In [ ]:
# alarm probability
# False=[0.999, 0.71, 0.06, 0.05], True=[0.001, 0.29, 0.94, 0.95]

cpd_alarm = TabularCPD(
    variable="Alarm",
    variable_card=2,
    values=[[0.999, 0.71, 0.06, 0.05], [0.001, 0.29, 0.94, 0.95]],
    evidence=["Burglary", "Earthquake"],
    evidence_card=[2, 2],
)

Finally, let's create the CPDs for the 'JohnCalls' and 'MaryCalls' nodes.

In [ ]:
# JohnCalls probability
cpd_johncalls = TabularCPD(
    variable="JohnCalls",
    variable_card=2,
    values=[[0.95, 0.1], [0.05, 0.9]],
    evidence=["Alarm"],
    evidence_card=[2],
)

# MaryCalls probability
cpd_marycalls = TabularCPD(
    variable="MaryCalls",
    variable_card=2,
    values=[[0.9, 0.3], [0.1, 0.7]],
    evidence=["Alarm"],
    evidence_card=[2],
)

We can now add all the CPDs to the network.

In [ ]:
alarm_model.add_cpds(
    cpd_burglary, cpd_earthquake, cpd_alarm, cpd_johncalls, cpd_marycalls
)

Let's now do some checks to verify that our network is valid.

In [ ]:
# Print cpds, this is equivalent to the table created above with False=0, True=1
print(alarm_model.get_cpds('Alarm'))

In [ ]:
# Check if the cpds are valid for the model, hence all columns sums up to 1 
alarm_model.check_model()

In [ ]:
# Check independencies
# ⟂ = variables are independent
# | = given an evidence

alarm_model.get_independencies()

From the above code we for example get that:
- `(Burglary ⟂ Earthquake)`:  these two events are _independent_ from each other
- `(Burglary ⟂ MaryCalls, JohnCalls | Alarm)`: _given_ the alarm ringing, a burglary is _independent_ from whether John or Mary will call.


**Question**

Given the meaning of `⟂ ` and `|`, what does `(JohnCalls ⟂ Earthquake, MaryCalls | Burglary, Alarm)` mean? 

**Inference**

Let's now query our network. In pmgpy there are different inference methods, we will use **BeliefPropagation**, one of the very basic methods of inference in graphical models, which consist of following the network edges and compute the corresponding probabilities.

In [ ]:
from pgmpy.inference import BeliefPropagation

# initialise the inference engine
alarm_infer = BeliefPropagation(alarm_model)

To query our inference engine we will use `.query` that takes two main parameters: a list of _variables_, and a dictionary of  _evidence_. For example for _P(a|b=0)_, a is our variable and b=0 the evidence.

We want to answer the following question:
- **Q1:** how likely is that Mary will call?
- **Q2:** given John called, what is the probability that the an earthquake trigerred the alarm?
- **Q3 (from lecture notes):** What is the probability that alarm has sounded, but neither a burglary nor an earthquake has occurred, and both John and Mary call?


In [ ]:
# Q1: how luckily is that Mary will call?
# We need to compute P(MaryCalls)

q1 = alarm_infer.query(variables=['MaryCalls'])
print(q1)

To fully answer our question we need to access the value in `MaryCalls(1)`

In [ ]:
q1.values[1]

For the _axiom of probability_, P(MaryCalls) is the sum of the probabilities of all the possible events (refer to the _probability model slides from Week5 lecture_).

In [ ]:
# Q2: given John called, what is the probability that the an earthquake trigerred the alarm?
# We need to compute P(Earthquake=True, Alarm=True | JohnCalls=True)

q2 = alarm_infer.query(variables=['Earthquake', 'Alarm'], evidence={'JohnCalls':1})
print(q2)

The answer is given by the entry `Earthquake(1), Alarm(1)`.

In [ ]:
# We can also get the most likely situation given some variables and evidences 
q2_most_likely = alarm_infer.map_query(variables=['Earthquake', 'Alarm'], evidence={'JohnCalls':1})
print(q2_most_likely)

In [ ]:
# Q3: What is the probability that alarm has sounded, 
# but neither a burglary nor an earthquake has occurred, and both John and Mary call?

# We need to compute P(Brglary=False,Eartquake=False, Alarm=True, MaryCalls=True, JohnCalls=True)

q3 = alarm_infer.query(variables=['JohnCalls', 'MaryCalls', 'Alarm', 'Burglary','Earthquake'])
print(q3)

The answer is given by the entry `JohnCalls(1) , MaryCalls(1) , Alarm(1) , Burglary(0) , Earthquake(0)`.

### Exercise 1 <a class="anchor" id="pgmpy-ex1"></a>

pmgpy allows you to import/export .bif files, also known as Bayesian Interchange Format files, these are specific file format used to represent Bayesian networks.





In [ ]:
from pgmpy.readwrite import BIFReader

You have been provided wih .bif file called `asia.bif`. The network represents the process for diagnosing patients arriving at a clinic. Each node within the network represents a specific patient condition; for instance, "asia" signifies whether the patient has recently traveled to Asia; 'smoke' is linked to 'lung cancer' and 'bronchitis', etc.

Let's read the file into memory.

In [ ]:
asia = BIFReader('./asia.bif').get_model()
print(asia)

**Task1**

Visualise the network using NetworkX. What are the nodes?

In [ ]:
# write here your code

**Task2**

What is the CPD table for 'bronchitis'?

In [ ]:
# write here your code

**Task3**

Answer the following questions:

* **Q1:** what is the probability that the patient has dyspnea?
* **Q2:** given the patient is a smoker, is it more likely they have bronchitis or dyspnea?

In [ ]:
# write here your code

### Exercise 2 <a class="anchor" id="pgmpy-ex2"></a>

So far we have worked with Bayesian networks already knowing the conditional probabilities associated to a set of variables. Given a set of sample data, how can we construct the corrresponding Bayesian network?

Creating a Bayesian network from a sample of data can be split into two problems:

- **Parameter learning**:  estimate the (conditional) probability distributions of the individual variables.

- **Structure learning**: capture the dependencies between the variables.


#### Parameter learning

Let's create some data

In [ ]:
import pandas as pd
data = pd.DataFrame(data={'fruit': ["banana", "apple", "banana", "apple", "banana","apple", "banana",
                                    "apple", "apple", "apple", "banana", "banana", "apple", "banana",],
                          'tasty': ["yes", "no", "yes", "yes", "yes", "yes", "yes",
                                    "yes", "yes", "yes", "yes", "no", "no", "no"],
                          'size': ["large", "large", "large", "small", "large", "large", "large",
                                    "small", "large", "large", "large", "large", "small", "small"]})
print(data)

From the dataset it is clear that 'tasty' depends from 'fruit' and 'size'. So we can create the folllowing network:

In [ ]:
fruit_model = BayesianNetwork([('fruit', 'tasty'), ('size', 'tasty')]) 

We want to estimate the values of the conditional probability distributions (CPDs), for the variables fruit, size, and tasty.

A natural estimate for the CPDs is to simply count the frequencies, with which the variable states occure in the dataset. This is called [Maximum Likelihood Estimation (MLE)](https://en.wikipedia.org/wiki/Maximum_likelihood_estimation). This approach allows to create CPDs in a such way that P(data|fruit_model) is maximal.

In [ ]:
from pgmpy.estimators import MaximumLikelihoodEstimator

# estimate CPDs using MLE
mle = MaximumLikelihoodEstimator(fruit_model, data)

In [ ]:
# Estimate CPD for the variable 'tasty'
print(mle.estimate_cpd('tasty'))

**Task1**

 What is the probabily that a small apple is tasty?

In [ ]:
# write here your code

**Task2**

Modify the dataset (adding more 'fruit' entries, or modify the existing one). What is the CPD for tasty in the new model?

In [ ]:
# write here your code

#### Structure learning

In the example above we have set the model structure. however, with a big dataset, this might not always be possible.

There are different techniques in pmgpy that we can use to learn the structure of the dataset. In what follows we will use the [hill climbing search](https://en.wikipedia.org/wiki/Hill_climbing), and we will measure the fit between model and data using the [Bayesian Information Criterion (BIC)](https://en.wikipedia.org/wiki/Bayesian_information_criterion) score.

In [ ]:
# Createa random sample of data
import numpy as np
data = pd.DataFrame(np.random.randint(0, 3, size=(2500, 8)), columns=list('ABCDEFGH'))

# Let's assume that values in A are the some of B & C, and H is given by the difference between G and A
data['A'] += data['B'] + data['C']
data['H'] = data['G'] - data['A']

In [ ]:
from pgmpy.estimators import HillClimbSearch, BicScore

# compute the 'best' model edges
hc = HillClimbSearch(data)
best_model = hc.estimate(scoring_method=BicScore(data))
print(best_model.edges())

The estimation produces edges among the A,B,C,G,H nodes leaving out all the others given the values are random and independent.

**Task1**

Create a Bayesian network from the `best_model` and visualise the network in NetworkX.

In [ ]:
# write here your code

**Task2**

Using a Bayesian Estimator (`from pgmpy.estimators import BayesianEstimator`) estimates the CPDs for each variable. What is the probability of H given G?

In [ ]:
# write here your code